In [2]:
import pandas as pd
#need to pip install pyxlsb

## Création de la base de donnée relatives aux résultats de l'élection communale de 2020
Pour des fins de comparabilité, nous avons choisit de concentrer notre analyse sur les communes de France métropolitaine (donc sans les données des DROMs, COMs, Terres australes et de Nouvelle Calédonie). Pour ce faire, nous avons récupéré la base de donnée des résultats à laquelle nous avons rajouté la nuance politique da la liste élue par *merging* avec la base de donnée des candidats. Cette donnée n'est disponible que pour les communes de plus de 3 500 habitants 

In [15]:
#on fait le choix d'enlever les DROM, les COM, les Terres australes et la Nouvelle-Calédonie : code de département None ou supérieurs à 900
#le département de corse du sud sera noté 201 au lieu de 2A et haute corse 202 au lieu de 2B
elus_2020 = pd.read_parquet('/home/onyxia/work/Python-project/Data/elus-conseillers-municipaux-cm.parquet')
elus_2020['Code du département'] = elus_2020['Code du département'].replace('2A', '201')
elus_2020['Code du département'] = elus_2020['Code du département'].replace('2B', '202')
elus_2020['Code du département'] = pd.to_numeric(elus_2020['Code du département'], errors='coerce').astype('Int64')
elus_2020 = elus_2020.dropna(subset=['Code du département'])
elus_2020 = elus_2020[elus_2020['Code du département']<900]
elus_2020 = elus_2020[elus_2020['Libellé de la fonction']=='Maire']

elus_2020 = elus_2020.drop(columns=['Code de la collectivité à statut particulier', 'Libellé de la collectivité à statut particulier','Libellé de la fonction'])

elus_2020.head(10)

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat,Date de début de la fonction,Code nationalité
3,1,Ain,01001,L'Abergement-Clémenciat,BOULON,Daniel,M,1951-04-03,74,Ancien cadre,2020-05-18,2020-05-26,FR
21,1,Ain,01002,L'Abergement-De-Varey,ORSET,Max,M,1947-02-11,65,"Ouvrier qualifié de la manutention, du magasin...",2020-05-18,2020-05-27,FR
37,1,Ain,01004,Ambérieu-En-Bugey,FABRE,Daniel,M,1961-07-09,74,Ancien cadre,2020-05-18,2020-05-28,FR
60,1,Ain,01005,Ambérieux-En-Dombes,FORNES,Christine,F,1962-06-07,21,Artisan,2021-02-05,2021-07-05,FR
77,1,Ain,01006,Ambléon,BIONDA,Annie,F,1951-11-28,77,Ancien employé,2020-05-18,2020-05-25,FR
95,1,Ain,01007,Ambronay,MANCUSO,Vincent,M,1960-02-07,74,Ancien cadre,2023-03-26,2023-03-31,FR
110,1,Ain,01008,Ambutrix,DELOFFRE,Dominique,M,1962-03-10,37,Cadre administratif et commercial d'entreprise,2020-05-18,2020-05-25,FR
123,1,Ain,01009,Andert-Et-Condon,GUILLERMIN,Patrick,M,1965-07-20,22,Commerçant et assimilé,2020-05-18,2020-05-28,FR
131,1,Ain,01011,Apremont,DOCHE,Claude,M,1943-12-16,74,Ancien cadre,2020-06-28,2020-06-28,FR
143,1,Ain,01012,Aranc,MATHIEU,Daniel,M,1948-02-04,74,Ancien cadre,2020-06-28,2020-05-07,FR


In [4]:
candidats_2020 = pd.read_excel('/home/onyxia/work/Python-project/Data/livre-des-listes-et-candidats-2020.xlsb', header=2)

In [ ]:
candidats_2020['Code du département'] = candidats_2020['Code du département'].replace('2A', '201')
candidats_2020['Code du département'] = candidats_2020['Code du département'].replace('2B', '202')
candidats_2020['Code du département'] = pd.to_numeric(candidats_2020['Code du département'], errors='coerce').astype('Int64')
candidats_2020 = candidats_2020.dropna(subset=['Code du département'])
candidats_2020 = candidats_2020[candidats_2020['N° candidat']==1]

candidats_2020 = candidats_2020[['Code du département', 'Code commune','Libellé commune', 'Nuance Liste', 'Nom candidat', 'Prénom candidat' ]]
candidats_2020['Code commune'] = candidats_2020['Code du département'].astype(str).str.zfill(2) + candidats_2020['Code commune'].astype(str).str.zfill(3)

candidats_2020.head(10)

In [26]:
elec_2020 = pd.merge(
    left=elus_2020, 
    right=candidats_2020,
    how='inner',
    left_on=['Code de la commune', "Nom de l'élu", "Prénom de l'élu"],
    right_on=['Code commune', "Nom candidat", "Prénom candidat"] 
)

elec_2020 = elec_2020.drop(columns=['Code sexe', 'Date de naissance', 'Code de la catégorie socio-professionnelle', 'Libellé de la catégorie socio-professionnelle',
'Date de début du mandat', 'Date de début de la fonction', 'Code nationalité', 'Code du département_y', 'Code de la commune', 'Libellé de la commune',
'Nom candidat', "Prénom candidat"])
elec_2020.rename(columns={"Code du département_x": "Code département"})

elec_2020.head(10)

,Code du département_x,Libellé du département,Nom de l'élu,Prénom de l'élu,Code commune,Libellé commune,Nuance Liste
0,1,Ain,FABRE,Daniel,01004,Ambérieu-en-Bugey,LDVC
1,1,Ain,MANCUSO,Vincent,01007,Ambronay,NaN
2,1,Ain,CRACCHIOLO,Philippe,01014,Arbent,NaN
3,1,Ain,PACCAUD,Richard,01021,Ars-sur-Formans,NaN
4,1,Ain,DESCHAMPS,Roland,01022,Artemare,NaN
5,1,Ain,MARTIN,Walter,01024,Attignat,NaN
6,1,Ain,BERNIGAUD,Christian,01025,Bâgé-Dommartin,LDIV
7,1,Ain,MEANT,Patrick,01027,Balan,NaN
8,1,Ain,RAVET,Véronique,01031,Bellignat,LDVG
9,1,Ain,PETIT,Régis,01033,Valserhône,LDVD
